# Natural Language Understanding

Starts with some more useful regex patterns:

`r"\bme\b"` will match only the word "me"  
`[A-Z]{1}[a-z]*` will match any title case word.  

If you're going to use a pattern several times, then store it with `re.compile()`.

Use of pipe operators within a pattern to match several, also use of `pattern.findall()` for multiple matches within a sentence.

***
## Flexibly match intents

In [1]:
import re
import time

In [2]:
intent_dict = {
    'goodbye': ['see ya', 'bye'],
    'greet': [r'\bhi\b', 'hola', 'heya'],
    'thankyou': ['appreciate', 'thank', r'\bta\b']
 }

In [3]:
# compile a dict of regex patterns that can look for any of the above pattern matches
intent_patterns = {}
for key, values in intent_dict.items():
    multi_pat = "|".join(values)
    compiled_pat = re.compile(multi_pat)
    # label this flexible pattern with the intent key it came with
    intent_patterns[key] = compiled_pat
intent_patterns

{'goodbye': re.compile(r'see ya|bye', re.UNICODE),
 'greet': re.compile(r'\bhi\b|hola|heya', re.UNICODE),
 'thankyou': re.compile(r'appreciate|thank|\bta\b', re.UNICODE)}

In [4]:
# Define a function to find the intent of a message
def find_intent(pat_dict, some_input):
    matched = None
    for intent, patterns in pat_dict.items():
        # Check for a pattern match first
        if patterns.search(some_input):
            matched = intent
    return matched

In [5]:
print(find_intent(intent_patterns, "hola! Como estas"))
print(find_intent(intent_patterns, "thankee sai"))
print(find_intent(intent_patterns, "see ya bud!"))

greet
thankyou
goodbye


***
## Respond to the intent

In [6]:
response_dict = {
    'default': '...',
    'goodbye': 'Have a great day',
    'greet': 'Hi there',
    'thankyou': "no problem, that's my job"
 }

In [7]:
def answer(string_input, pat_dict, resps):
    # get the matched intent
    intent = find_intent(pat_dict, string_input.lower())
    # Use default as the fll back value
    key = "default"
    if intent in resps:
        key = intent
    return resps[key]

In [8]:
answer("See ya", intent_patterns, response_dict)

'Have a great day'

Update the wrapper function from module 1 that uses a nice display template.

In [9]:
# update params to include the lookup dicts as default
def user_speaks(user_input, pat_dict=intent_patterns, resps=response_dict, user_format="USER:", bot_format="BOT:"):
    """Passes the user's input to response handler."""
    time.sleep(0.6)
    print(f"{user_format} {user_input}")
    # update the line below to use the new flexible match functions
    resp = answer(user_input, pat_dict, resps)
    time.sleep(0.6)
    return f"{bot_format} {resp}"

In [10]:
print(user_speaks("Hi hi cherry pie!"))
print(user_speaks("Ta very much my lovely..."))
print(user_speaks("Gotta go. Bye bye hunny pie."))

USER: Hi hi cherry pie!
BOT: Hi there
USER: Ta very much my lovely...
BOT: no problem, that's my job
USER: Gotta go. Bye bye hunny pie.
BOT: Have a great day


***
## Basic NER

Named Entity Recognition. 

In [11]:
def get_names(string_input):
    """Searches a string for an indication that a name is being discussed, then search
    for a proper noun and return it if found."""
    # ensure None is returned if no match is found
    entity = None
    name_pat = re.compile("name|call")
    proper_noun_pat = re.compile("[A-Z]{1}[a-z]*")
    # look for a sentence about a named entity:
    if name_pat.search(string_input):
        entity = proper_noun_pat.findall(string_input)
        if len(entity) > 0:
            # several hits means we need to concatenate values
            entity = " ".join(entity)
    return entity

In [12]:
print(get_names("my name is Jimmy."))
print(get_names("My name is Jimmy."))
# you can see how this would be limited and won't work with lowering an input string

Jimmy
My Jimmy


In [13]:
# Define respond()
def answer_name(str_input):
    name = get_names(str_input)
    if name is None:
        return "You're mysterious, tell me your name."
    else:
        return f"Hello, {name}!"

In [14]:
# update params to include the lookup dicts as default
def user_speaks(user_input, pat_dict=intent_patterns, resps=response_dict, user_format="USER:", bot_format="BOT:"):
    """Passes the user's input to response handler."""
    time.sleep(0.6)
    print(f"{user_format} {user_input}")
    # update the line below to use the name retrieval funcs
    resp = answer_name(user_input)
    time.sleep(0.6)
    return f"{bot_format} {resp}"

In [15]:
print(user_speaks("i am called John Snow"))
print(user_speaks("my name is Spartacus"))
print(user_speaks("My name is Spartacus"))

USER: i am called John Snow
BOT: Hello, John Snow!
USER: my name is Spartacus
BOT: Hello, Spartacus!
USER: My name is Spartacus
BOT: Hello, My Spartacus!


***

## Wordvec with spaCy

Great little intro on word vectors, where tokens - floats - are assigned to words, word parts, letters or sentences. These can then be used within ML workflows. spaCy makes several wordvec models available. Here we are using `en_core_web_sm` which is trained upon a large corpus with the GloVe algorithm.

Tokens can be compared to others using their cosine similarity:

* Vector directions point in same direction = 1
* Vector directions are perpindicular = 0 
* Vector directions are opposite = -1

In [16]:
import spacy
nlp = spacy.load('en_core_web_md')

/opt/anaconda3/envs/py-chatbots/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
n_dim = nlp.vocab.vectors_length
n_dim

300

In [18]:
# Use the nlp model on a string to get tokens:
doc = nlp("Hey Ho, ah let's go!")
doc

Hey Ho, ah let's go!

In [19]:
for token in doc:
    print(f"{token}: {token.vector[:7]}")
# showing the first 7 word vectors for the sentence tokens

Hey: [ 2.9      0.48218 -2.2693   0.27522 -7.1124   1.2409  -0.43371]
Ho: [-1.9577  -3.629   -4.1803   0.75524  2.439    4.1769  -1.2797 ]
,: [-3.3899  -4.7034  -0.56101  1.2291   4.3298  -1.0775  -1.3006 ]
ah: [ 3.5059   2.9413  -0.30366 -0.53069 -3.0985   3.9806  -2.8103 ]
let: [ 8.0705   6.2403  -5.6268  -0.6813  -3.603    2.8543  -0.82774]
's: [ 3.3163   9.7209  -3.1254  -5.1013  12.248    0.74676 -2.2017 ]
go: [ 1.484   8.3944 -8.3806  3.2081 -4.2582  1.9773 -2.7806]
!: [ 5.0891  -3.3753  -4.2695  -4.8156   3.8904   6.2171   0.26271]


In [20]:
len(doc[0].vector) == n_dim
# you can see that each token has n_dim

True

In [21]:
# download ATIS dataset

import pandas as pd
import requests
import json
#URL = 'https://raw.githubusercontent.com/jkkummerfeld/text2sql-data/master/data/atis.json'
#data = json.loads(requests.get(URL).text)
## Flattening JSON data
#ATIS = pd.json_normalize(data)
#ATIS.head()
# This source did not have the labels from what I could see, so going with kaggle instead


In [26]:
import os
from pyprojroot import here
# there is a kaggle api, but overkill for this project. Source is:
# https://www.kaggle.com/datasets/hassanamin/atis-airlinetravelinformationsystem?resource=download
colnames = ["label", "text"]
ATIS = pd.read_csv(os.path.join(here(), "data", "atis_intents_train.csv"),
names = colnames)
sentences = ATIS.text
n_sent = len(sentences)
ATIS.head()

,label,text
0,atis_flight,i want to fly from boston at 838 am and arriv...
1,atis_flight,what flights are available from pittsburgh to...
2,atis_flight_time,what is the arrival time in san francisco for...
3,atis_airfare,cheapest airfare from tacoma to orlando
4,atis_airfare,round trip fares from pittsburgh to philadelp...


In [30]:
# prepare a 2D array for storing the vectors
import numpy as np
X_train = np.zeros((n_sent, n_dim))
print(np.shape(X_train))
X_train

(4834, 300)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [31]:
# pass all the sentences to spacy to calclate the word vectors, storing in our array
for row, sentence in enumerate(sentences):
    doc = nlp(sentence)
    X_train[row, :] = doc.vector
X_train


array([[-0.0826674 , -0.58716798, -2.1964817 , ..., -1.25990617,
        -2.6429491 ,  1.24336207],
       [-1.80790913,  1.00080669, -3.05921483, ..., -1.52719319,
        -2.36632752,  1.1796701 ],
       [-1.36993992,  0.49113077,  0.07254934, ..., -1.3198179 ,
        -2.0484488 ,  2.16480947],
       ...,
       [-1.82230973, -1.06588328, -2.15213466, ..., -2.93899107,
        -2.00138211, -1.13957   ],
       [-2.00526071,  2.67443204, -2.5772748 , ..., -0.08352997,
        -2.17978454,  0.47018856],
       [-1.33432257,  3.42474079, -2.36559343, ...,  0.34180367,
        -3.09680247,  2.61888719]])

***

## Use an SVM to recognise intents

Various approaches to cosine similarity are discussed, but the exercises are about fitting a support vector classifier to our data. Data is labelled and with a train test split as is usual, so prepare this now.

In [32]:
# encode labels as numeric
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
print(ATIS.label)
le.fit(ATIS.label)
y_train = le.transform(ATIS.label)
y_train

0            atis_flight
1            atis_flight
2       atis_flight_time
3           atis_airfare
4           atis_airfare
              ...       
4829        atis_airfare
4830         atis_flight
4831        atis_airline
4832         atis_flight
4833         atis_flight
Name: label, Length: 4834, dtype: object


array([4, 4, 5, ..., 3, 4, 4])

***

## Prep the Test Data

Import and prep the test data

In [36]:
ATIS_test = pd.read_csv(
    os.path.join(here(), "data", "atis_intents_test.csv"), names=colnames)
ATIS_test.head()

,label,text
0,atis_flight,i would like to find a flight from charlotte ...
1,atis_airfare,on april first i need a ticket from tacoma to...
2,atis_flight,on april first i need a flight going from pho...
3,atis_flight,i would like a flight traveling one way from ...
4,atis_flight,i would like a flight from orlando to salt la...


In [51]:
test_sentences = ATIS_test.text
n_test_sent = len(test_sentences)
X_test = np.zeros((n_test_sent, n_dim))
# create the word vectors
for row, sentence in enumerate(test_sentences):
    doc = nlp(sentence)
    X_test[row, :] = doc.vector
X_test

array([[-1.52020073,  2.37595367, -3.60137939, ...,  0.99911278,
        -3.60158348,  1.67940843],
       [-1.04470658,  0.23952016, -3.36406946, ..., -0.71620345,
        -2.85818815,  1.2970823 ],
       [-1.11312497,  2.18487287, -3.68941283, ..., -1.3552922 ,
        -3.2125392 ,  0.5888651 ],
       ...,
       [-1.85663342,  3.25489068, -4.70206499, ...,  1.86979628,
        -5.62847376,  0.69890302],
       [-1.73054779,  3.26366663, -3.23943543, ...,  0.01536671,
        -4.43544579,  2.05688596],
       [-0.18429269,  2.82947516, -3.51153922, ...,  1.13745189,
        -3.99869776,  1.25371277]])

***

## Predict Labels

In [59]:
from sklearn.svm import SVC
clf = SVC()
# fit the model to train data
clf.fit(X_train, y_train)

SVC(C=1)

In [60]:
y_pred = clf.predict(X_test)
y_pred[:10]

array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4])

Translate the numeric class labels back to text ad check for accuracy.

In [61]:
pred_labels = list(le.inverse_transform(y_pred))
pred_labels[:10]

['atis_flight',
 'atis_flight',
 'atis_flight',
 'atis_flight',
 'atis_flight',
 'atis_flight',
 'atis_flight',
 'atis_flight',
 'atis_flight',
 'atis_flight']

In [64]:
correct = sum(pred_labels == ATIS_test.label)
tot = len(ATIS_test.label)
f"Predicted {correct} labels correctly from a total of {tot} rows"
f"Accuracy: {round(correct / tot * 100, 2)} %"

'Accuracy: 96.25 %'